In [2]:
!pip install --quiet git+https://github.com/huggingface/transformers sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Import the required libraries
from transformers import SeamlessM4TModel
import torch

# Load the pre-trained SeamlessM4T model from the 🤗 Transformers Hub
# model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")
model = SeamlessM4TModel.from_pretrained("facebook/seamless-m4t-v2-large")

# Check if CUDA is available, if yes, set the device to "cuda:0", else use the CPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Move the model to the specified device (CUDA if available, otherwise CPU)
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
You are using a model of type seamless_m4t_v2 to instantiate a model of type seamless_m4t. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of SeamlessM4TModel were not initialized from the model checkpoint at facebook/seamless-m4t-v2-large and are newly initialized: ['speech_encoder.encoder.layers.0.conv_module.batch_norm.bias', 'speech_encoder.encoder.layers.0.conv_module.batch_norm.num_batches_tracked', 'speech_encoder.encoder.layers.0.conv_module.batch_norm.running_mean', 'speech_encoder.encoder.layers.0.conv_module.batch_norm.running_var', 'speech_encoder.encoder.layers.0.conv_module.batch_norm.weight', 'speech_encoder.encoder.layers.1.conv_module.batch_norm.bias', 'speech_encoder.encoder.layers.1.conv_module.batch_norm.num_batches_tracked', 'speech_encoder.encoder.layers.1.conv_module.batch_norm.running_mean', 'speech_encoder.encoder.layers.1.conv_module.batch_norm.running_var', 'speech_encoder.encoder.layers.1.conv_module.batch_norm.weight', 'speech_encoder.encoder.layers.10.conv_module.batch_norm.bias', 'speech_encoder.encoder.layers.10.conv_module.batch_norm.num_batches_tracked', 'speech_encoder.encod

In [4]:
# Import the necessary library for loading the AutoProcessor
from transformers import AutoProcessor

# Load the pre-trained SeamlessM4T medium checkpoint using the AutoProcessor
# processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")


# Extracting the sample rate from the model's configuration
sample_rate = model.config.sampling_rate

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Speech-to-Text Translation

In [5]:
!unzip '/content/archive.zip'

Archive:  /content/archive.zip
  inflating: harvard.wav             
  inflating: jackhammer.wav          


In [11]:
# Import the necessary libraries
import torchaudio

# Load the audio file
audio_sample, audio_sampling_rate = torchaudio.load("/content/harvard.wav")

# Check if the audio's sampling rate is different from the model's sampling rate and resample if necessary
if audio_sampling_rate != model.config.sampling_rate:
    audio_sample = torchaudio.functional.resample(audio_sample,
                                                  orig_freq=audio_sampling_rate,
                                                  new_freq=model.config.sampling_rate)

# Process the audio inputs using the specified processor, device, and sampling rate
audio_inputs = processor(audios=audio_sample, return_tensors="pt", sampling_rate=sample_rate).to(device)

# Generate text from the processed audio inputs, targeting French as the output language and disabling speech generation
output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)

# Decode the output tokens to obtain the translated text from the audio
translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

# Print the translated text obtained from the audio
print(f"Translated Text: {translated_text_from_audio}")

Translated Text: I don't know how to get a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this, a little bit of this


In [8]:
# Generate text from the processed audio inputs, targeting French as the output language and disabling speech generation
output_tokens = model.generate(**audio_inputs, tgt_lang="no", generate_speech=False)

# Decode the output tokens to obtain the translated text from the audio
translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

# Print the translated text obtained from the audio
print(f"Translated Text: {translated_text_from_audio}")

ValueError: `tgt_lang=no` is not supported by this model.
                    Please specify a `tgt_lang` in afr,amh,arb,ary,arz,asm,azj,bel,ben,bos,bul,cat,ceb,ces,ckb,cmn,cmn_Hant,cym,dan,deu,ell,eng,est,eus,fin,fra,fuv,gaz,gle,glg,guj,heb,hin,hrv,hun,hye,ibo,ind,isl,ita,jav,jpn,kan,kat,kaz,khk,khm,kir,kor,lao,lit,lug,luo,lvs,mai,mal,mar,mkd,mlt,mni,mya,nld,nno,nob,npi,nya,ory,pan,pbt,pes,pol,por,ron,rus,sat,slk,slv,sna,snd,som,spa,srp,swe,swh,tam,tel,tgk,tgl,tha,tur,ukr,urd,uzn,vie,yor,yue,zlm,zul. Note that SeamlessM4T supports
                    more languages for text translation than for speech synthesis.

In [7]:
# Generate text from the processed audio inputs, targeting French as the output language and disabling speech generation
output_tokens = model.generate(**audio_inputs, tgt_lang="hin", generate_speech=False)

# Decode the output tokens to obtain the translated text from the audio
translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

# Print the translated text obtained from the audio
print(f"Translated Text: {translated_text_from_audio}")

Translated Text: पुराने बीयर की सुगंध रहती है गंध को बाहर लाने के लिए गर्मी लगती है एक ठंडा डिप स्वास्थ्य और जस्ट को पुनर्स्थापित करती है एक नमकीन अचार का स्वाद हैम के साथ टकोज़ल पास्टोर मेरा पसंदीदा है एक जस्टफुल फूड है हॉट क्रॉस बून


In [8]:
# Generate text from the processed audio inputs, targeting French as the output language and disabling speech generation
output_tokens = model.generate(**audio_inputs, tgt_lang="nob", generate_speech=False)

# Decode the output tokens to obtain the translated text from the audio
translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

# Print the translated text obtained from the audio
print(f"Translated Text: {translated_text_from_audio}")

ValueError: `tgt_lang=nob` is not supported by this model.
                    Please specify a `tgt_lang` in arb,ben,cat,ces,cmn,cym,dan,deu,eng,est,fin,fra,hin,ind,ita,jpn,kan,kor,mlt,nld,pes,pol,por,ron,rus,slk,spa,swe,swh,tam,tel,tgl,tha,tur,ukr,urd,uzn,vie. Note that SeamlessM4T supports
                    more languages for text translation than for speech synthesis.

### Text-to-Speech Translation

In [ ]:
# Importing the Audio module for displaying the generated audio
from IPython.display import Audio

# Processing the text input
text_inputs = processor(text="Hello, How are you?", src_lang="eng", return_tensors="pt").to(device)

# Generating audio from the processed text
audio_array_from_text = model.generate(**text_inputs, tgt_lang="eng")[0].cpu().numpy().squeeze()

# Displaying the generated audio using IPython's Audio function
Audio(audio_array_from_text, rate=sample_rate)

### Text-to-Text Translation

In [18]:
# Processing the text input
text_inputs = processor(text="Hello, How are you?", src_lang="eng", return_tensors="pt").to(device)

# Generating text from the processed text
text_array = model.generate(**text_inputs, tgt_lang="nno", generate_speech=False)

print(f"Translated Text:- {processor.decode(text_array[0].tolist()[0], skip_special_tokens=True)}")

ValueError: `tgt_lang=nno` is not supported by this model.
                    Please specify a `tgt_lang` in arb,ben,cat,ces,cmn,cym,dan,deu,eng,est,fin,fra,hin,ind,ita,jpn,kan,kor,mlt,nld,pes,pol,por,ron,rus,slk,spa,swe,swh,tam,tel,tgl,tha,tur,ukr,urd,uzn,vie. Note that SeamlessM4T supports
                    more languages for text translation than for speech synthesis.

### Speech-to-Speech Translation

In [ ]:
import torchaudio

# Load the audio file
audio_sample, audio_sampling_rate = torchaudio.load("/content/download.wav")

# Resample the audio if the sampling rate is different from the model's sampling rate
if audio_sampling_rate != model.config.sampling_rate:
    audio_sample = torchaudio.functional.resample(audio_sample,
                                                  orig_freq=audio_sampling_rate,
                                                  new_freq=model.config.sampling_rate)

# Process the audio inputs
audio_inputs = processor(audios=audio_sample,
                         return_tensors="pt",
                         sampling_rate=sample_rate).to(device)

# Generate speech from the processed audio inputs
audio_array_from_audio = model.generate(**audio_inputs, tgt_lang="rus")[0].cpu().numpy().squeeze()

# Displaying the generated audio using IPython's Audio function
Audio(audio_array_from_audio, rate=sample_rate)

## Tips and Tricks

### Use dedicated models

In [11]:
# Import the SeamlessM4TForSpeechToSpeech model from the transformers library
from transformers import SeamlessM4TForSpeechToSpeech

# Load the SeamlessM4TForSpeechToSpeech model
model = SeamlessM4TForSpeechToSpeech.from_pretrained("facebook/hf-seamless-m4t-medium")

### Modify the speaker's identity

In [12]:
import torchaudio
from IPython.display import Audio

# Load the audio file
audio_sample, audio_sampling_rate = torchaudio.load("/content/harvard.wav")

# Resample the audio if the sampling rate is different from the model's sampling rate
if audio_sampling_rate != model.config.sampling_rate:
    audio_sample = torchaudio.functional.resample(audio_sample,
                                                  orig_freq=audio_sampling_rate,
                                                  new_freq=model.config.sampling_rate)

# Process the audio inputs
audio_inputs = processor(audios=audio_sample, return_tensors="pt",sampling_rate=sample_rate).to(device)

# Generate speech from the processed audio inputs
audio_array_from_audio = model.generate(**audio_inputs, tgt_lang="rus",spkr_id=7)[0].cpu().numpy().squeeze()

# Displaying the generated audio using IPython's Audio function
Audio(audio_array_from_audio, rate=sample_rate)

### Modify generation strategy

In [14]:
# Processing the text input
text_inputs = processor(text="Hello, How are you?", src_lang="eng", return_tensors="pt").to(device)

# Generating text from the processed text
text_array = model.generate(**text_inputs,
                            tgt_lang="hin",
                            generate_speech=False,
                            text_num_beams=4,
                            speech_do_sample=True)

print(f"Translated Text:- {processor.decode(text_array[0].tolist()[0], skip_special_tokens=True)}")

TypeError: object of type 'NoneType' has no len()

### Leveraging Batch Processing for Enhanced Efficiency

In [ ]:
# Processing the text input
text_inputs = processor(text=["Hello, how are you?", "I am fine, thank you."], src_lang="eng", return_tensors="pt").to(device)

# Generating text from the processed text
text_array = model.generate(**text_inputs,
                            tgt_lang="hin",
                            generate_speech=False,
                            text_num_beams=4,
                            speech_do_sample=True)

print(f"Sentence 1:- {processor.decode(text_array[0].tolist()[0], skip_special_tokens=True)}")
print(f"Sentence 2:- {processor.decode(text_array[0].tolist()[1], skip_special_tokens=True)}")

### Generate both speech and text

In [ ]:
# Processing the text input
text_inputs = processor(text="Hello, How are you?", src_lang="eng", return_tensors="pt").to(device)

# Generating audio/text from the processed text
audio_text_output = model.generate(**text_inputs,
                            tgt_lang="rus",
                            text_num_beams=4,
                            speech_do_sample=True,
                            return_intermediate_token_ids=True)

# Displaying the generated audio using IPython's Audio function
audio_array = audio_text_output[0].cpu().numpy().squeeze()
Audio(audio_array, rate=sample_rate)

text_array = audio_text_output[2]
translated_text_from_text = processor.decode(text_array.tolist()[0], skip_special_tokens=True)
print(f"Translated Text: {translated_text_from_text}")